In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

# Boundary Value Problem

I will work through the basic approach to solving a boundary value problem (BVP) for ordinary differential equations (ODEs).  Assuming a second order ODE for the variable $y(t)$, an *initial value problem* (IVP) specifies values for $y$ and $dy/dt$ at $t=t_{0}$.  A BVP instead specifies values for $y$ at times $t=t_{0}$ and $t=t_{1}$ which we will label $y_{0}$ and $y_{1}$.

Of course, we still need to solve the ODE, which requires an initial value for $dy/dt$, which we have to guess (or at best estimate) and then vary.  The only available general approach to solving BVPs computationally is to perform a root search: we need to find the zeros of the quantity $y(t_{1}) - y_{1}$ as we vary $\frac{dy}{dt}(t_{0})$.  The actual process is (relatively) straightforward; it's just understanding how it works that may be complicated.

We'll start with a simple manual search, and then show how to automate it.  We'll consider the problem of calculating the height, $z$, of a projectile (say a ball of some kind) moving under the influence of gravity, and we will simply specify that the height of the ball at both times $t_{0}$ and $t_{1}$ should be zero; the ball is not constrained (i.e. there is no ground to hit, to avoid the trivial solution $z=0\, \forall \,t$).

## Setting up the ODE solver

Assuming that there is only motion in the $z$ direction, we have the following pair of coupled equations:
$$\frac{dz}{dt} = v$$
$$\frac{dv}{dt} = -g$$
where we will assume the standard value $g = 9.81ms^{-2}$.  The first thing to do is set up a function to return the right-hand sides of these coupled equations as a single array.

In [2]:
def ball_vertical_RHS(t,y):
    """Calculate RHS for projectile ODE
    
    Inputs:
    t  Time (not used, but here for interface)
    y  Array of z and v
    """
    g = 9.81 # m/s^2
    z = y[0]
    v = y[1]
    dz = v
    dv = -g # Gravity acts downwards
    return np.array([dz,dv])

In [3]:
# Set up initial conditions
t0 = 0.0 #s
t1 = 10.0 #s
m = 1 # kg

# Boundary conditions that we want to fulfil
z0 = 0.0 # m
z1 = 0.0 # m 

Now we'll explore how we can solve the problem manually.  To solve for the final position of the projectile, we will use the SciPy `solve_ivp` function from the `integrate` module.  The return object (`y0` in the code below) stores positions and velocities in the array `y` for all timesteps; we access the position or velocity with the first index, and the timestep with the second index.  As we want the position and the final time, we use `y[0,-1]`. Let's assume a possible but not very sensible first velocity, $v_{0}=0.0$, and see what happens:

In [4]:
v0 = 0.0 # m/s
y0 = np.array([z0,v0])
dist0 = integrate.solve_ivp(ball_vertical_RHS,(t0,t1),y0)
print(f"With initial velocity {v0}m/s at t={t1}s we have z={dist0.y[0,-1]}m")

With initial velocity 0.0m/s at t=10.0s we have z=-490.4999999999999m


This makes physical sense, since there is no initial velocity and constant negative acceleration.  Let's try a positive value of $v_0$.

In [5]:
v0 = 50.0 # m/s
y0 = np.array([z0,v0])
dist0 = integrate.solve_ivp(ball_vertical_RHS,(t0,t1),y0)
print(f"With initial velocity {v0}m/s at t={t1}s we have z={dist0.y[0,-1]}m")

With initial velocity 50.0m/s at t=10.0s we have z=9.500000000000213m


That is much better, though now we need to reduce the initial velocity.  Notice how we're using $v_{0}$ as the independent variable; we might define $f(v_{0}) = z_{1}(v_{0}) - z_{1\, \mathrm{required}}$ as the function whose root we want to find.  We could now refine our solution manually, but that won't teach us anything, so I won't go any further.

## Automating the search

Let's define a python function which we can use in the root finding process.

In [6]:
def projectile_final_height_error(v0,z1):
    """Function to allow search for final height of projectile
    by returning the difference between the actual height calculated
    with initial velocity v0 and desired height z1.
    
    Assumes that t0, t1 and z0 are defined externally.
    
    Inputs:
    v0  Starting velocity
    z1  Target final height
    
    Output:
    zerr  Value of z1(v0) - z1
    """
    z0 = 0.0 # m
    y0 = np.array([z0,v0])
    
    # Solve for the final position of the ball
    dist0 = integrate.solve_ivp(ball_vertical_RHS,(t0,t1),y0)
    
    # Error in final position
    err = dist0.y[0,-1] - z1
    
    return err

And now we will implement a root-finding procedure to discover the correct initial velocity, $v_{0}$, to give the desired height $z_{1}$ at $t_{1}$.  Here we'll use bisection for simplicity and clarity.  We'll use the first two values for $v_{0}$ that we found above manually as brackets.

In [7]:
# Target height
z1 = 0.0

# Brackets worked out manually above
v0_1 = 0.0
f_1 = projectile_final_height_error(v0_1,z1)
v0_2 = 50.0
f_2 = projectile_final_height_error(v0_2,z1)
print(f"Brackets are ({v0_1},{v0_2}) with heights of ({f_1},{f_2})")

# Parameters
max_iter = 100
iter = 0
tol = 1e-3
f_mid = 10*tol # Just to ensure that we enter the while loop

while abs(f_mid)>tol and iter<max_iter:
    # Simple bisection
    v0_mid = 0.5*(v0_1 + v0_2)
    f_mid = projectile_final_height_error(v0_mid,z1)
    if f_mid*f_1>0:
        v0_1 = v0_mid
        f_1 = f_mid
    else:
        v0_2 = v0_mid
        f_2 = f_mid
    iter = iter+1
print(f"Finished after {iter} iterations")
print(f"Initial velocity of {v0_mid}m/s gives final height of {f_mid}m")

Brackets are (0.0,50.0) with heights of (-490.4999999999999,9.500000000000213)
Finished after 18 iterations
Initial velocity of 49.049949645996094m/s gives final height of -0.0005035400388777589m
